# Prediction Seniority: Fine-Tuning a Classification Model



In [24]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, classification_report

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
import os


In [25]:
# we do this so the github preview works
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

In [26]:
# GitHub raw helper
GH_USER = "luisadosch"
GH_REPO = "Final-Project-snapAddy"
BRANCH = "main"


def get_github_url(relative_path):
    return f"https://raw.githubusercontent.com/{GH_USER}/{GH_REPO}/{BRANCH}/{relative_path}"

In [27]:
jobs_annotated = pd.read_csv(get_github_url("data/processed/jobs_annotated.csv"))
df_seniority = pd.read_csv(get_github_url("data/raw/seniority-v2.csv"))

# test set from annotated CVs (ACTIVE only) -> out-of-production set
jobs_annotated_df = jobs_annotated.loc[
    jobs_annotated["status"] == "ACTIVE",
    ["position", "seniority"]
].copy()

jobs_annotated_df = jobs_annotated_df.rename(columns={"position": "text", "seniority": "label"})

# keep only (text, label)
df_seniority = df_seniority[["text", "label"]].copy()
jobs_annotated_df = jobs_annotated_df[["text", "label"]].copy()

print("fine tuning data:", len(df_seniority), "CV labeled data:", len(jobs_annotated_df))
print("fine-tune label counts:\n", df_seniority["label"].value_counts())
print("out-of-production label counts:\n", jobs_annotated_df["label"].value_counts())


fine tuning data: 9428 CV labeled data: 623
fine-tune label counts:
 label
Senior        3733
Lead          3546
Director       984
Management     756
Junior         409
Name: count, dtype: int64
out-of-production label counts:
 label
Professional    216
Management      192
Lead            125
Senior           44
Director         34
Junior           12
Name: count, dtype: int64


## Data Setup

We use `df_seniority` as the supervised fine-tuning dataset (training distribution).
We use `jobs_annotated_df` as a separate, real-world evaluation set (production distribution)
to estimate out-of-production generalization error.

However, we are aware that there is the label `Professional` in the `jobs_annotated_df` which does not exist in `df_seniority`. Therefore we run our fine-tuning on a regression model.


## Train / Validation / Test split (fine-tuning dataset)

* We split the fine-tuning dataset into train/val/test.
* Train is used to update weights, validation is used for early stopping,
and the held-out test provides an in-distribution estimate.

In [28]:
train_df_base, temp_df = train_test_split(
    df_seniority,
    test_size=0.30,              # 70% train, 30% temp
    stratify=df_seniority["label"],
    random_state=42,
)

val_df_base, df_seniority_test = train_test_split(
    temp_df,
    test_size=0.50,              # 15% val, 15% test
    stratify=temp_df["label"],
    random_state=42,
)

print("train:", len(train_df_base), "val:", len(val_df_base), "test:", len(df_seniority_test))
print("train label counts:\n", train_df_base["label"].value_counts())
print("val label counts:\n", val_df_base["label"].value_counts())
print("test label counts:\n", df_seniority_test["label"].value_counts())

train: 6599 val: 1414 test: 1415
train label counts:
 label
Senior        2613
Lead          2482
Director       689
Management     529
Junior         286
Name: count, dtype: int64
val label counts:
 label
Senior        560
Lead          532
Director      147
Management    113
Junior         62
Name: count, dtype: int64
test label counts:
 label
Senior        560
Lead          532
Director      148
Management    114
Junior         61
Name: count, dtype: int64


In [29]:
ORD_MAP = {
    "Junior": 1.0,
    "Professional": 2.0,
    "Senior": 3.0,
    "Lead": 4.0,
    "Management": 5.0,
    "Director": 6.0,
}
INV_ORD = {v: k for k, v in ORD_MAP.items()}

In [30]:
def add_synthetic(train_df: pd.DataFrame, synthetic_csv_relpath: str) -> pd.DataFrame:
    syn = pd.read_csv(get_github_url(synthetic_csv_relpath))
    syn = syn[["position", "seniority"]].copy()

    id2label = {v: k for k, v in ORD_MAP.items()}
    syn["label"] = syn["seniority"].map(id2label)
    syn = syn.rename(columns={"position": "text"})
    syn = syn.dropna(subset=["text", "label"])

    out = pd.concat([train_df[["text", "label"]], syn[["text", "label"]]], ignore_index=True)
    return out

Tokenizer + Tokenization

In [31]:

MODEL_CKPT = "xlm-roberta-base"
MAX_LEN = 48

tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
    )

In [32]:
def to_hf_dataset(df: pd.DataFrame, label_col: str = "labels") -> Dataset:
    ds = Dataset.from_pandas(df[["text", label_col]].copy(), preserve_index=False)
    ds = ds.map(tokenize, batched=True)
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", label_col])
    return ds

In [33]:
def eval_split(trainer, train_ds, val_ds, test_ds):
    train_metrics = trainer.evaluate(train_ds, metric_key_prefix="train")
    val_metrics   = trainer.evaluate(val_ds,   metric_key_prefix="val")
    test_metrics  = trainer.evaluate(test_ds,  metric_key_prefix="test")
    print("Evaluation based on training data:", train_metrics)
    print("Evaluation based on val data:  ", val_metrics)
    print("Evaluation based on test data: ", test_metrics)
    return train_metrics, val_metrics, test_metrics

In [34]:
def loss_curves(trainer):
    logs = pd.DataFrame(trainer.state.log_history)

    train_loss = logs.dropna(subset=["loss"])[["step", "loss"]].copy()
    eval_loss  = logs.dropna(subset=["eval_loss"])[["step", "eval_loss"]].copy()

    import matplotlib.pyplot as plt

    plt.figure()
    plt.plot(train_loss["step"], train_loss["loss"])
    plt.xlabel("step")
    plt.ylabel("train loss")
    plt.title("Train loss")
    plt.show()

    plt.figure()
    plt.plot(eval_loss["step"], eval_loss["eval_loss"])
    plt.xlabel("step")
    plt.ylabel("eval loss")
    plt.title("Eval loss")
    plt.show()

    return logs

# Regression Fine-tuning (Ordinal Targets)

We map ordered seniority classes to a numeric scale and fine-tune a transformer with a regression head.
This avoids a hard failure when certain classes (e.g., 'Professional') are missing from the production-like CV dataset.
Evaluation reports include MAE and thresholded accuracy/F1 by converting predicted scores back into label bins. We do this because our other models are evaluated with accuracy, and this way it is easier for us to compare the different models.

In [35]:


def score_to_label(s: float) -> str:
  # because we used accuracy for other classification models, we also want to get an accuracy here
    if s < 1.5: return "Junior"
    if s < 2.5: return "Professional"
    if s < 3.5: return "Senior"
    if s < 4.5: return "Lead"
    if s < 5.5: return "Management"
    return "Director"

def compute_metrics_reg(eval_pred):
    preds, labels = eval_pred
    scores = np.squeeze(preds)
    mae = mean_absolute_error(labels, scores)

    y_pred = [score_to_label(float(s)) for s in scores]
    y_true = [INV_ORD[float(int(round(x)))] for x in labels]

    return {
        "mae": mae,
        "acc_thresh": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
    }




In [36]:
def predict_table_reg(trainer, ds, df_text_label):
    pred = trainer.predict(ds)
    scores = np.squeeze(pred.predictions)

    out = df_text_label.copy().reset_index(drop=True)
    out["pred_score"] = scores
    out["pred_label"] = [score_to_label(float(s)) for s in scores]
    out["abs_err"] = (out["pred_score"] - out["label"].map(ORD_MAP).astype(float)).abs()
    return out.sort_values("abs_err", ascending=False)

In [37]:
def run_regression(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame, out_dir: str = "seniority_ft_reg"):
    tr = train_df.copy()
    va = val_df.copy()
    te = test_df.copy()

    tr["labels"] = tr["label"].map(ORD_MAP).astype(float)
    va["labels"] = va["label"].map(ORD_MAP).astype(float)
    te["labels"] = te["label"].map(ORD_MAP).astype(float)

    train_ds = to_hf_dataset(tr, label_col="labels")
    val_ds   = to_hf_dataset(va, label_col="labels")
    test_ds  = to_hf_dataset(te, label_col="labels")

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CKPT,
        num_labels=1,
        problem_type="regression",
    )

    args = TrainingArguments(
        output_dir=out_dir,
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=10,
        weight_decay=0.05,
        warmup_ratio=0.06,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="mae",
        greater_is_better=False,
        fp16=True,
        report_to="none",
        dataloader_num_workers=0,
        logging_strategy="steps",
        logging_steps=50,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_reg,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train()

    eval_split(trainer, train_ds, val_ds, test_ds)
    loss_curves(trainer)

    train_pred = predict_table_reg(trainer, train_ds, tr[["text", "label"]])
    val_pred   = predict_table_reg(trainer, val_ds,   va[["text", "label"]])
    test_pred  = predict_table_reg(trainer, test_ds,  te[["text", "label"]])

    display(test_pred.head(30))
    return trainer, (train_pred, val_pred, test_pred)



## Classification Fine-tuning (Discrete Labels)

Here we later fine-tune a multi-class classifier on the seniority labels (because later we use additional labels so we don't have the missing label problem anymore).
However, even with the additional data, we only have a small number of 'Professional' labels. To reduce class imbalance, we oversample the training split only.
Validation is used for early stopping; the held-out test estimates in-distribution performance.
Finally, we evaluate on CV data to estimate out-of-production error.

In [38]:
def compute_metrics_cls(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }


In [39]:
def predict_table_cls(trainer, ds, df_text_label, id2label):
    pred = trainer.predict(ds)
    probs = pred.predictions
    y_pred = np.argmax(probs, axis=-1)

    out = df_text_label.copy().reset_index(drop=True)
    out["pred_id"] = y_pred
    out["pred_label"] = [id2label[int(i)] for i in y_pred]
    out["correct"] = (out["pred_label"] == out["label"])
    return out

In [40]:
def report_cls(trainer, ds, id2label):
    pred = trainer.predict(ds)
    y_pred = np.argmax(pred.predictions, axis=-1)
    y_true = pred.label_ids
    print(classification_report(y_true, y_pred, target_names=[id2label[i] for i in range(len(id2label))]))

In [41]:
from sklearn.utils import resample

def oversample_df(df, label_col="label", random_state=42):
    max_n = df[label_col].value_counts().max()
    parts = []
    for lab, g in df.groupby(label_col):
        parts.append(resample(g, replace=True, n_samples=max_n, random_state=random_state))
    return pd.concat(parts).sample(frac=1, random_state=random_state).reset_index(drop=True)

def run_classification(train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame, out_dir: str = "seniority_ft_cls"):
    # label space ONLY from train_df (no leakage)
    label_list = sorted(train_df["label"].unique())
    label2id = {l: i for i, l in enumerate(label_list)}
    id2label = {i: l for l, i in label2id.items()}

    tr = train_df.copy()
    va = val_df.copy()
    te = test_df.copy()

    tr["labels"] = tr["label"].map(label2id).astype(int)
    va["labels"] = va["label"].map(label2id)
    te["labels"] = te["label"].map(label2id)

    print("labels:", label_list)
    print("val missing labels:", int(va["labels"].isna().sum()))
    print("test missing labels:", int(te["labels"].isna().sum()))

    # drop rows with unseen labels (important for out-of-production where labels may differ)
    va = va.dropna(subset=["labels"]).copy()
    te = te.dropna(subset=["labels"]).copy()
    va["labels"] = va["labels"].astype(int)
    te["labels"] = te["labels"].astype(int)

    # oversample train only
    train_os = oversample_df(tr, label_col="label")
    train_ds = to_hf_dataset(train_os, label_col="labels")
    val_ds   = to_hf_dataset(va,       label_col="labels")
    test_ds  = to_hf_dataset(te,       label_col="labels")

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CKPT,
        num_labels=len(label2id),
        id2label=id2label,
        label2id=label2id,
    )

    args = TrainingArguments(
        output_dir=out_dir,
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=10,
        weight_decay=0.05,
        warmup_ratio=0.06,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1_macro",
        greater_is_better=True,   # IMPORTANT: higher F1 is better
        fp16=True,
        report_to="none",
        dataloader_num_workers=0,
        logging_strategy="steps",
        logging_steps=50,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_cls,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train()

    eval_split(trainer, train_ds, val_ds, test_ds)
    loss_curves(trainer)

    train_pred = predict_table_cls(trainer, train_ds, train_os[["text", "label"]], id2label)
    val_pred   = predict_table_cls(trainer, val_ds,   va[["text", "label"]],      id2label)
    test_pred  = predict_table_cls(trainer, test_ds,  te[["text", "label"]],      id2label)

    display(test_pred.head(30))
    report_cls(trainer, val_ds, id2label)

    return trainer, (train_pred, val_pred, test_pred), (label2id, id2label)


## Model Runs

## Data and evaluation design

We use two different datasets with two different purposes:

1. `df_seniority` (fine-tuning distribution):  
   A curated dataset of job titles with seniority labels.  
   This dataset is split into train/validation/test to support proper model selection and in-distribution evaluation.

2. `jobs_annotated_df` (production / out-of-distribution dataset):  
   Real CV job titles from our pipeline.  
   This dataset is **never used for training or early stopping**.  
   We only use it after fine-tuning to estimate **out-of-production generalization error** (distribution shift).

This setup mirrors deployment: the model is trained on a standard label dataset, but used on CV text that may differ in wording, noise, and label frequencies.

## Interpreting the outputs

We report two kinds of performance:

1. In-distribution performance (from `df_seniority_test`):  
   This tells us how well the model generalizes to the same labeling scheme and data style as the training data.

2. Out-of-distribution / production performance (from `jobs_annotated_df`):  
   This estimates real deployment error under distribution shift, such as:
   different vocabulary in CVs,
   shorter/noisier titles,
   different label frequencies,
   and sometimes labels not present in the training label space (for classification).

For OOD classification evaluation, we drop CV rows whose labels are not in the classifier label set to prevent invalid metrics.
We always report how many rows were dropped to keep the evaluation transparent.


## 1. Regression without synthetic data & without oversampling

In [ ]:
# In-distribution evaluation (df_seniority test split)
reg_trainer, (reg_train_pred, reg_val_pred, reg_test_pred) = run_regression(
    train_df_base,
    val_df_base,
    df_seniority_test,
    out_dir="seniority_ft_reg_base"
)

Map:   0%|          | 0/6599 [00:00<?, ? examples/s]

Map:   0%|          | 0/1414 [00:00<?, ? examples/s]

Map:   0%|          | 0/1415 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3962634583.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


In [ ]:
# Out-of-production evaluation (CV dataset)

reg_cv_pred = predict_table_reg(
    reg_trainer,
    to_hf_dataset(jobs_annotated_df.assign(labels=jobs_annotated_df["label"].map(ORD_MAP).astype(float)), label_col="labels"),
    jobs_annotated_df[["text", "label"]]
)

In [ ]:
display(reg_cv_pred.head(30))
print("CV threshold-accuracy:", (reg_cv_pred["pred_label"] == reg_cv_pred["label"]).mean())

## Performance of 1. Regression without synthetic data & without oversampling

## Classification with Synthetic Data & Oversampling

## Why synthetic augmentation + classification

After establishing a stable baseline with regression, we also train a classification model.
To reduce the missing/rare-class problem (especially `Professional`), we augment the training set with synthetic samples.

Because class frequencies remain imbalanced even after augmentation, we oversample **only the training split**.
This keeps validation/test unbiased and prevents leaking distribution changes into model selection.


Gemini -> because then we also have label professional -> we use oversampling to get better representation of professional

In [ ]:
train_df_aug = add_synthetic(train_df_base, "data/results/gemini_synthetic.csv")

In [ ]:
cls_trainer_syn, (cls_train_pred_syn, cls_val_pred_syn, cls_test_pred_syn), (label2id_syn, id2label_syn) = run_classification(
    train_df_aug,
    val_df_base,
    df_seniority_test,
    out_dir="seniority_ft_cls_syn"
)

In [ ]:
# show how often the different labels occur in train_df_aug

train_df_aug["label"].value_counts()

In [ ]:
# Out-of-distribution eval for classification model
cv_cls = jobs_annotated_df[["text", "label"]].copy()

# map string labels -> class ids of the SYNTHETIC-trained classifier
cv_cls["labels"] = cv_cls["label"].map(label2id_syn)

# drop rows whose labels are not in the classifier label space
cv_cls = cv_cls.dropna(subset=["labels"]).copy()
cv_cls["labels"] = cv_cls["labels"].astype(int)

cv_ds = to_hf_dataset(cv_cls, label_col="labels")

cv_pred = predict_table_cls(
    cls_trainer_syn,
    cv_ds,
    cv_cls[["text", "label"]],
    id2label_syn
)

print("CV accuracy:", cv_pred["correct"].mean())


In [ ]:
display(cv_pred.head(30))